##### IMPORTAR AS PLANILHAS

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

##### IMPORTANDO PLANILHA

In [ ]:
path_iw = "../../../data/util/os/"

df_iw49 = pd.read_csv(path_iw + "IW49_Executadas_processado.csv")

In [ ]:
df_iw49["Data fim"] = pd.to_datetime(df_iw49["Data fim"])
df_iw49["Data inicio"] = pd.to_datetime(df_iw49["Data inicio"])
df_iw49["Local de instalação"] = df_iw49["Local de instalação"].astype(str)

##### CRIANDO DATAFRAME SEM INCLUIR DATAS FINAIS ANTERIORES A DATAS INICIAIS

In [ ]:
# Removendo linhas onde a data final é superior a data inicial
df_sem_linhas_irregulares = df_iw49.loc[df_iw49["Data fim"]
                                        >= df_iw49["Data inicio"]]

num_linhas_removidas = df_iw49.shape[0] - df_sem_linhas_irregulares.shape[0]
print("Linhas removidas:", num_linhas_removidas)

##### FUNÇÃO QUE ADD COLUNA DE HH POR MES E QUANTIDADE DE MESES PASSADOS DURANTE A OS

In [ ]:
# Soma dos meses, em funcao da quantidade de anos que se passaram e meses, entre o inicio e fim da OS


def duracaoHH(linha):
    linha["Meses"] = (
        (linha["Data fim"].year - linha["Data inicio"].year) * 12
        + (linha["Data fim"].month - linha["Data inicio"].month)
    ) + 1
    linha["HH por mes"] = linha["HH final"] / linha["Meses"]
    return linha


df = df_sem_linhas_irregulares.apply(duracaoHH, axis=1)

##### FUNCAO QUE AGRUPA OS POR MES, ANO, LOCAL

In [ ]:
# Criação da coluna PK para criar strings do conjunto de instalação e ano mês, para facilitar a verificação de existencia de OS naquela instalação, mês e ano

df_agrupado = pd.DataFrame(
    columns=["Ano Mes", "HH", "Local de instalação", "PK"])

for index, row in df.iterrows():
    data_inicio = row["Data inicio"].replace(day=1)
    data_fim = row["Data fim"].replace(day=1)
    date_range = pd.date_range(start=data_inicio, end=data_fim, freq="MS")

    hh_por_mes = row["HH por mes"]
    local_instalacao = row["Local de instalação"]

    for date in date_range:
        ano_mes = date.strftime("%Y-%m")

        if ano_mes + local_instalacao in df_agrupado["PK"].values:
            df_agrupado.loc[
                (df_agrupado["Ano Mes"] == ano_mes)
                & (df_agrupado["Local de instalação"] == local_instalacao),
                "HH",
            ] += hh_por_mes  # Somando HH apos agrupamento por local, ano e mes

        else:
            df_agrupado = pd.concat(
                [
                    df_agrupado,
                    pd.DataFrame(
                        {
                            "Ano Mes": [ano_mes],
                            "Local de instalação": [local_instalacao],
                            "HH": [hh_por_mes],
                            "PK": [ano_mes + local_instalacao],
                        }
                    ),
                ]
            )

df_agrupado = df_agrupado.sort_values(by="Ano Mes").reset_index(drop=True)
df_agrupado = df_agrupado.drop(columns=["PK"])

##### EXPORTANDO PLANILHA

In [ ]:
df_agrupado.to_csv(path_iw + "os_entrada.csv", index=False)